<a href="https://colab.research.google.com/github/hkaragah/risk_reliability/blob/main/Monte_Carlo_Simulation_Roulette_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Simulation: Roulette Simulator

In [2]:
# Fair Roulette has 36 numbers (from 1 to 36) all from the same color (simplified) and all has the same chance of winning a prize
class FairRoulette():
  def __init__(self):
    self.pockets = []
    for i in range(1,37):
      self.pockets.append(i)
    self.ball = None
    self.pocketOdds = len(self.pockets) - 1

  def spin(self):
    self.ball = random.choice(self.pockets)

  def betPocket(self, pocket, amt):
    if str(pocket) == str(self.ball):
      return amt * self.pocketOdds
    else:
      return -amt

  def __str__(self):
    return 'Fair Roulette'

# European Roulette has one zero (0) in addition to those 36 numbers in Fair Roulette which has no chance of winning a prize (unfair)
class EuRoulette(FairRoulette):
  def __init__(self):
    FairRoulette.__init__(self)
    self.pockets.append('0')

  def __str__(self):
    return 'European Roulette'


# American Roulette has one double-zero (00) in addition to those 37 numbers (1 through 36 and 0) in European Roulette which has no chance of winning a prize (more unfair)
class AmRoulette(EuRoulette):
  def __init__(self):
    EuRoulette.__init__(self)
    self.pockets.append('00')

  def __str__(self):
    return 'American Roulette'

In [22]:
from IPython.display import HTML

# Your image URL
image_url = 'https://betandslots.com/wp-content/uploads/2020/09/american-roulette-european.jpg'

# Create the HTML string to embed the image
html_str = f'<img src="{image_url}" style="width:600px; height:auto;" alt="Embedded Image"/>'


# Display the image in the notebook
HTML(html_str)


In [3]:
def playRoulette(game, numSpins, pocket, bet, toPrint):
  totPocket = 0
  for i in range(numSpins):
    game.spin()
    totPocket += game.betPocket(pocket, bet)
  if toPrint:
    print(numSpins, 'spins of', game)
    print('Expected return betting', pocket, '=', str(100*totPocket/numSpins) + '%\n')
  return (totPocket/numSpins)

In [5]:
import random

random.seed(0)
game = EuRoulette()
for numSpins in (100, 1_000_000):
  for i in range(3):
    playRoulette(game, numSpins, 2, 1, True)

100 spins of European Roulette
Expected return betting 2 = -100.0%

100 spins of European Roulette
Expected return betting 2 = 44.0%

100 spins of European Roulette
Expected return betting 2 = -28.0%

1000000 spins of European Roulette
Expected return betting 2 = -2.8%

1000000 spins of European Roulette
Expected return betting 2 = -2.0512%

1000000 spins of European Roulette
Expected return betting 2 = -2.296%



In [16]:
def findPocketReturn(game, numTrials, trialSize, toPrint):
  pocketReturns = []
  for t in range(numTrials):
    trialVals = playRoulette(game, trialSize, 2, 1, toPrint)
    pocketReturns.append(trialVals)
  return pocketReturns

In [17]:
def getMeanAndStd(X):
  try:
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
      tot += (x - mean)**2
    std = (tot/len(X))**0.5
    return mean, std
  except ZeroDivisionError:
    print('Zero Division Error')
  except TypeError:
    print('Type Error')


### Empirical Rule

Under these assumtions:
* The mean estimation error is zero (no bias in errors), it is a valid assumption in most simulations, but it could be different in a labratory experiment where depending on the technique, there might be a bias in one direction causing the mean of error to becomes skewed in one direction.
* Distribution of errors is normal


The following is valid:
* ~68% of data within one standard deviation of mean
* ~95% of data within one standard 1.96 deviation of mean
* ~99.7% of data within one standard 3 deviation of mean


In [19]:
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette)

for G in games:
  resultDict[G().__str__()] = []

for numSpins in (100, 1000, 1_000_000):
  print('\nSimulating', numSpins, 'spins of each game')
  for G in games:
    pocketRetuns = findPocketReturn(G(), 20, numSpins, False)
    mean, std = getMeanAndStd(pocketRetuns)
    resultDict[G().__str__()].append((numSpins, 100*mean, 100*std))
    print('Exp. return for', G(), '=', str(round(100*mean, 3)) + '%', '+/-' + str(round(100*1.96*std, 3)) + '% with 95% confidence')


Simulating 100 spins of each game
Exp. return for Fair Roulette = 2.6% +/-118.648% with 95% confidence
Exp. return for European Roulette = 8.0% +/-115.942% with 95% confidence
Exp. return for American Roulette = -29.8% +/-90.567% with 95% confidence

Simulating 1000 spins of each game
Exp. return for Fair Roulette = -9.64% +/-37.86% with 95% confidence
Exp. return for European Roulette = -1.18% +/-39.379% with 95% confidence
Exp. return for American Roulette = -1.54% +/-33.638% with 95% confidence

Simulating 1000000 spins of each game
Exp. return for Fair Roulette = -0.061% +/-1.22% with 95% confidence
Exp. return for European Roulette = -2.694% +/-1.091% with 95% confidence
Exp. return for American Roulette = -5.426% +/-0.966% with 95% confidence


## References:

1. <a href="https://youtu.be/OgO1gpXSUzU?si=FIRFfLZPfyn8ixXR">MIT OpenCourseWare Cannel, 6.Monte Carlo Simulation</a>